In [1]:
#!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies

In [2]:
import torch
from io import BytesIO
from PIL import Image
%pylab inline 
import cv2
from IPython.display import clear_output
from sort import *
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


Using cache found in /home/cmt/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-4-1 Python-3.10.9 torch-1.12.0+cu102 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.eval()
Tensor = torch.cuda.FloatTensor

In [4]:
def detect_image(img, img_size=416):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0).to(device)
    #input_img = Variable(image_tensor.type(Tensor))
    # run inference on the model and get detections
    with torch.no_grad():
        detections = model(image_tensor)
        #detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    return detections



In [5]:
videopath = 'etc/vidroom2.mp4'



cmap = plt.get_cmap('tab20b')
colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

# initialize Sort object and video capture
vid = cv2.VideoCapture(videopath)

# Save output to avi video
fourcc = cv2.VideoWriter_fourcc(*"XVID")
fps = 25.0
width = 1080
height = 1920
out = cv2.VideoWriter("output.avi", fourcc, fps, (width, height))
mot_tracker = Sort(max_age=int(fps)) 

#while(True):
counter = 0
while vid.isOpened():
    print("Frame ", counter)
    ret, frame = vid.read()
    if frame is None:
        print('Finished!')
        out.release()
        break
    counter += 1
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.flip(frame, 0)
    #pilimg = Image.fromarray(frame)
    detections = model(frame)
    #print("N det: ", len(detections.pandas().xyxy[0]))


    #detections.render()  # updates results.ims with boxes and labels
    #for im in detections.ims:
    #    buffered = BytesIO()
    #    im_base64 = Image.fromarray(im)
    #frame = cv2.cvtColor(numpy.array(im_base64), cv2.COLOR_RGB2BGR)
    if detections is not None:
        print("N det: ", len(detections.pandas().xyxy[0]))
        tracked_objects = mot_tracker.update(detections.pandas().xyxy[0].iloc[:,:5].to_numpy(), detections.pandas().xyxy[0].name.values)
        #tracked_objects = mot_tracker.update(detections.pandas().xyxy[0].to_numpy())
        print("N tracked: ", len(tracked_objects))

        for x1, y1, x2, y2, obj_id, obj_class in tracked_objects:
            x1, y1, x2, y2, obj_id = int(float(x1)), int(float(y1)), int(float(x2)), int(float(y2)), int(obj_id)

            color = colors[int(obj_id) % len(colors)]
            color = [i * 255 for i in color]
            #cv2.rectangle(frame, (x1, y1), (x1+box_w, y1+box_h), color, 4)
            #cv2.rectangle(frame, (x1, y1-35), (x1+len(str(obj_id))*19+60, y1), color, -1)
            #cv2.putText(frame, str(obj_id), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
            
            # Draw the bounding box around the tracked object
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.rectangle(frame, (x1, y1), (x1+len(obj_class+' - '+str(obj_id))*18, y1-25), color, -1)
            cv2.putText(frame, obj_class+' - '+str(obj_id), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    out.write(frame)
    clear_output(wait=True)


Frame  643
Finished!
